# トレーニングスクリプトをコマンドジョブとして実行する

Python SDK for Azure Machine Learningを使用すると、スクリプトをコマンドジョブとして送信できます。ジョブを使用することで、機械学習モデルをトレーニングする際の入力パラメータと出力を簡単に追跡できます。

## Before you start

このノートブックのコードを実行するには、**azure-ai-ml**パッケージの最新版が必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、識別子のパラメータ（サブスクリプションID、リソースグループ名、ワークスペース名）が必要です。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、ワークスペースに接続するにはデフォルト値を使用できる。

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## コマンドを実行する

糖尿病を予測する分類モデルをトレーニングするために、以下のセルを実行します。モデルは**src**フォルダにある**train-model-parameters.py**スクリプトを実行することで学習されます。学習データとして**diabetes.csv**ファイルを使用します。

- code`: 実行するスクリプトを含むフォルダを指定します。
- command`: 具体的に何を実行するかを指定する。
- 環境`: コマンドを実行する前に計算機にインストールするパッケージを指定する。
- compute`: コマンドの実行に使用するコンピュートを指定する。
- display_name`: 個々のジョブの名前。
- experiment_name`: ジョブが属する実験の名前。

ターミナルでスクリプトをテストするためのコマンドは、以下のジョブの設定にあるコマンドと同じであることに注意してください。

In [4]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",# コマンドでpyファイルの実行を行っている
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.53 MBs): 100%|███████

Monitor your job at https://ml.azure.com/runs/ivory_jelly_gm1fck00ll?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


In [ ]:
# 